In [2]:
# Make sure the Colab Runtime is set to Accelerator: TPU.
import requests
import os
backend = 'gpu'
if backend =='tpu':
  # The following is required to use TPU Driver as JAX's backend.
  from jax.config import config
  config.FLAGS.jax_xla_backend = "tpu_driver"
  config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
  print(config.FLAGS.jax_backend_target)

import jax.numpy as jnp
import jax
print(jnp.asarray([1,2,3]).device_buffer.device())

cpu:0


In [3]:
def setup(N,float_t):
    def _hfinterp_code1(histogramssets, alphasets):
        allset_allhisto_deltas_up = jnp.divide(histogramssets[:,:,2], histogramssets[:,:,1])
        allset_allhisto_deltas_dn = jnp.divide(histogramssets[:,:,0], histogramssets[:,:,1])
        
        allsets_allhistos_masks = jnp.where(alphasets > 0, jnp.ones(alphasets.shape), jnp.zeros(alphasets.shape))
        bases_up = jnp.einsum('sa,shb->shab', jnp.ones(alphasets.shape), allset_allhisto_deltas_up)
        bases_dn = jnp.einsum('sa,shb->shab', jnp.ones(alphasets.shape), allset_allhisto_deltas_dn)
        exponents = jnp.einsum('sa,shb->shab', jnp.abs(alphasets), jnp.ones(allset_allhisto_deltas_up.shape))
        masks = jnp.einsum('sa,shb->shab', allsets_allhistos_masks, jnp.ones(allset_allhisto_deltas_up.shape))

        bases = jnp.where(masks, bases_up, bases_dn)
        return jnp.power(bases, exponents)

    
    a_shape = (100,1)
    h_shape = (100,100,3,N)
    f = jax.jit(_hfinterp_code1)
    return f, [a_shape,h_shape]



In [4]:
f,shapes = setup(1000,0.2)

In [5]:
import numpy as np
a = jnp.asarray(np.random.uniform(size = shapes[0]))
h = jnp.asarray(np.random.uniform(size = shapes[1]))
print(a.device_buffer.device(),h.device_buffer.device())
print(f(h,a).shape,f)#trigger jit

cpu:0 cpu:0
(100, 100, 1, 1000) <function _python_jit.<locals>.f_jitted at 0x7f2f85b0a6a8>


In [6]:
%%timeit
f(h,a)

100 loops, best of 3: 131 ms per loop
